## Summary

Analyse Squirrel Finance Telegram Chat.

## Libs

In [1]:
import json

In [80]:
import numpy as np
import pandas as pd

## Inits

In [157]:
df = pd.read_json('../data/raw/channel_messages.json')

## Explore and Clean

In [158]:
# sort by message id
df.sort_values('id', inplace=True)
df.reset_index(drop=True, inplace=True)

In [159]:
# pickle
df.to_pickle('../data/interim/squirrelfinancetelegram.pkl')

In [27]:
df['_'].unique()
# 2 types of messages. MessageService is by Bot or Announcement

array(['MessageService', 'Message'], dtype=object)

In [32]:
# rename _ to message type
df.rename(columns={"_": "msg_type"}, inplace=True)

In [49]:
df['peer_id'].astype(str).unique()
# only 1 type of value which is returning the channel id
# hence can drop or replace with channel id

array(["{'_': 'PeerChannel', 'channel_id': 1190829614}"], dtype=object)

In [51]:
df.drop('peer_id', axis=1, inplace=True)

In [98]:
df.from_id.head(3)
# gives user object.
# better to parse and return the user id

0                                        None
1     {'_': 'PeerUser', 'user_id': 481914851}
2    {'_': 'PeerUser', 'user_id': 1289707287}
Name: from_id, dtype: object

In [99]:
# parse from_id
df['from_id'] = df.from_id.apply(
    lambda x: str(x['user_id']) if pd.notnull(x) else np.nan)

In [124]:
# is always null can drop
df[df.entities.notnull()]['entities'].head()

2    []
3    []
4    []
5    []
6    []
Name: entities, dtype: object

In [125]:
df.drop('entities', axis=1, inplace=True)

- 'mentioned' are those replies with @ (true/false)
- 'silent' has True/False values. Only MessageService has true.
- 'post' seems to be always false
- 'from_scheduled' is nan or 0. (perhaps is scheduled message)
- 'legacy' only has false
- 'edit_hide' is nan or 0. (perhaps is if user edited message)
- 'pinned' is nan or 0 or 1. (for user pinned msg is 1. nan is for messageService
- 'from_id' changed to user_id
- 'fwd_from' 
- 'via_bot_id' from the bot id
- 'reply_to'
- 'media'
- 'reply_markup' - keyboard button
- 'entities' - always null or blank list
- 'views' - view count
- 'forwards' - forward count
- 'replies' - whether it was replied to (json object)
- 'edit_date' - date of edit
- 'post_author' - seems to be null at the moment
- 'grouped_id' - not sure what id
- 'restriction_reason' - seems to be all emtpy list
- 'ttl_period' - seems to be all nan
- 'action' - message action like add user. Usually no message involved.

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15103 entries, 0 to 15102
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   msg_type            15103 non-null  object             
 1   id                  15103 non-null  int64              
 2   date                15103 non-null  datetime64[ns, UTC]
 3   message             13907 non-null  object             
 4   out                 15103 non-null  bool               
 5   mentioned           15103 non-null  bool               
 6   media_unread        15103 non-null  bool               
 7   silent              15103 non-null  bool               
 8   post                15103 non-null  bool               
 9   from_scheduled      13907 non-null  float64            
 10  legacy              15103 non-null  bool               
 11  edit_hide           13907 non-null  float64            
 12  pinned              13907 non-nu